<a href="https://colab.research.google.com/github/oscarmejias/deeplearning/blob/master/Stock_Market_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importamos las librerias**

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


##**Construimos la red neuronal de la AI del Trader**

In [0]:
class AI_Trader():
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"): #Manten, Compra, Vende
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen= 2000)
    self.inventory = []
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units= 32, activation= "relu", input_dim= self.state_size))

    model.add(tf.keras.layers.Dense(units= 64, activation= "relu"))

    model.add(tf.keras.layers.Dense(units= 128, activation= "relu"))

    model.add(tf.keras.layers.Dense(units= self.action_space, activation= "linear"))

    model.compile(loss= "mse", optimizer= tf.keras.optimizers.Adam(learning_rate= 0.001))

    return model

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):

    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      #print(len(self.memory) - batch_size + 1, len(self.memory)) # Curiosidad, borrar despues
      batch.append(self.memory[i])

    for state, action, reward, next_action, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        #print("esto es todo lo que arrojan todas las predicciones: {}".format(self.model.predict(next_state))) # Analizo el formato de lo que son las predicciones del modelo
        #print("esto es todo lo que arroja una prediccion: {}".format(self.model.predict(next_state)[0])) # Analizo el formato de lo que es una prediccion del modelo
        #print("este es el valor maximo de las predicciones: {}".format(np.amax(self.model.predict(next_state)[0]))) # Observo la prediccion mas alta del modelo

      target = self.model.predict(state)
      target[0][action] = reward

      #print("Estado: {} -- Objetivo: {}".format(state, target)) # Imprimo el state y target para familiarizarme con el algoritmo
      self.model.fit(state, target, epochs= 1, verbose= 0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay


##**Pre procesado del dataset**

**Definimos funciones adicionales**

In [0]:
#Defino la funcion sigmoide
def sigmoide(x):
  return 1/ (1 + math.exp(-x))

In [0]:
#Defino la funcion del formato de precios
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
#Defino la funcion que carga el Dataset
def dataset_loader(stock_name):

  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]

  close = dataset["Close"]

  return close

In [0]:
#FACEBOOK = data_reader.DataReader("FB", data_source="yahoo")
#FACEBOOK.head()

###**Creador de estados**


In [0]:
def state_creator(data, timestep, window_size):

  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoide(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

###**Cargando una divisa del mercado**

In [0]:
stock_name = "TSLA"
data = dataset_loader(stock_name)

In [9]:
data.head()

Date
2015-04-14    207.460007
2015-04-15    207.830002
2015-04-16    206.699997
2015-04-17    206.789993
2015-04-20    205.270004
Name: Close, dtype: float64

##**Entrenar la AI Trader**

**Configurar los hyper parametros**

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

**Definir el modelo del AI Trader**

In [0]:
trader = AI_Trader(window_size)

In [12]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


###**Bucle de entrenamiento**

In [0]:
for episode in range(1, episodes + 1):

  print("Episodio: {}/{}".format(episode, episodes))

  state = state_creator(data, 0, window_size + 1)

  total_profit = 0
  trader.inventory = []

  for t in tqdm(range(data_samples)):

    action = trader.trade(state)

    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0

    if action == 1: #Comprar
      trader.inventory.append(data[t])
      print("AI Trader compró: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: #Vender
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader Vendió: ", stocks_price_format(data[t]), "Beneficio: " + stocks_price_format(data[t] - buy_price))

    if t ==data_samples - 1:
      done = True
    else:
      done = False

    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print("#########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("#########################")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1257 [00:00<?, ?it/s]

Episodio: 1/1000
AI Trader compró:  $ 207.830002
AI Trader compró:  $ 205.270004
AI Trader compró:  $ 209.410004
AI Trader compró:  $ 219.440002
AI Trader Vendió:  $ 218.600006 Beneficio: $ 10.770004
AI Trader compró:  $ 231.550003
AI Trader Vendió:  $ 230.479996 Beneficio: $ 25.209991
AI Trader Vendió:  $ 226.050003 Beneficio: $ 16.639999
AI Trader Vendió:  $ 226.029999 Beneficio: $ 6.589996
AI Trader Vendió:  $ 230.509995 Beneficio: - $ 1.040009
AI Trader compró:  $ 232.949997
AI Trader Vendió:  $ 236.800003 Beneficio: $ 3.850006
AI Trader compró:  $ 248.839996
AI Trader Vendió:  $ 244.350006 Beneficio: - $ 4.489990
AI Trader compró:  $ 247.429993
AI Trader Vendió:  $ 250.800003 Beneficio: $ 3.370010


  3%|▎         | 37/1257 [00:15<46:39,  2.29s/it]

AI Trader compró:  $ 249.139999


  3%|▎         | 38/1257 [00:18<52:43,  2.60s/it]

...

AI Trader compró:  $ 227.009995


 82%|████████▏ | 1028/1257 [49:49<10:53,  2.85s/it]

AI Trader compró:  $ 232.309998


 82%|████████▏ | 1029/1257 [49:52<10:49,  2.85s/it]

AI Trader compró:  $ 231.949997
